In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 300)

import plotly.graph_objects as go
import plotly.express as px
%matplotlib inline

import os
import sys
import datetime
import time
from pathlib import Path

import folium
import geocoder

import warnings; warnings.filterwarnings('ignore')

sys.path.append('..')
from config import CFG
CFG = CFG()

date = datetime.datetime(2023, 3, 1).strftime('%Y-%m-%d')
CFG.DATE = date

In [2]:
PROCESSED_DATA_PATH = Path.cwd().parent / "data_files" / "processed"

In [3]:
df = pd.read_csv(PROCESSED_DATA_PATH / f"df-processed-{CFG.DATE}.csv")
latlng_df = pd.read_csv(PROCESSED_DATA_PATH / "geodata" / "latlng.csv", usecols=['city', 'lat', 'lng'])
#df['city'] = df['city'] + ', BC'
latlng_df

,city,lat,lng
0,"Vancouver, BC",49.26,-123.11
1,"Nelson, BC",49.49,-117.29
2,"Kamloops, BC",50.68,-120.34
3,"Surrey, BC",49.19,-122.85
4,"Kelowna, BC",49.89,-119.50
5,"Nanaimo, BC",49.16,-123.94
6,"Penticton, BC",49.50,-119.59
7,"Cranbrook, BC",49.51,-115.77
8,"Vernon, BC",50.27,-119.27
9,"New Westminster, BC",49.21,-122.91


In [4]:
# latlng_df[['city', 'lat', 'lng']].to_csv(PROCESSED_DATA_PATH / "geodata" / "latlng.csv"))

In [5]:
def get_latlng(city):
    g = geocoder.osm(city)
    lat = g.json['lat']
    lng = g.json['lng']
    return lat, lng

def latlng(cities):
    latlng_dict = {}
    for city in cities:
        sleepytime = np.random.rand() * 5
        time.sleep(sleepytime)
        lat, lng = get_latlng(city)
        latlng_dict[city] = [lat, lng]
    return latlng_dict

In [6]:
cities = df['city'].unique().tolist()
latlng_dict = latlng(cities)

counts = df.value_counts('city')

latlng = pd.DataFrame(latlng_dict).T.reset_index()
latlng.columns = ['city', 'lat', 'lng']
latlng_df = latlng.merge(counts.to_frame(), on='city').rename({0: 'count'}, axis=1)

latlng_df['text'] = latlng_df['city'] + '<br>' + 'Count: ' + latlng_df['count'].astype(str)
latlng_df = latlng_df.sort_values('count', ascending=False).reset_index(drop=True)

latlng_df['count_log'] = latlng_df['count'].apply(np.log)
# latlng_df.to_csv(PROCESSED_DATA_PATH / 'geodata' / "latlng.csv"), index=False)

In [7]:
cities

['Penticton, BC',
 'Vancouver, BC',
 'Nanaimo, BC',
 'Cranbrook, BC',
 'Kelowna, BC',
 'Terrace, BC',
 'Vernon, BC',
 'Chilliwack, BC',
 'Hope, BC',
 'New Westminster, BC',
 'Mission, BC',
 'Abbotsford, BC',
 'Kamloops, BC',
 'Burnaby, BC',
 'Surrey, BC',
 'Coquitlam, BC',
 'Grand Forks, BC',
 'Nelson, BC',
 'Castlegar, BC',
 'Prince George, BC',
 'Trail, BC',
 'Kimberley, BC',
 'Merritt, BC',
 'Princeton, BC',
 'Williams Lake, BC',
 'Powell River, BC',
 'Fairmont Hot Springs, BC',
 'Victoria, BC',
 'Boston Bar, BC',
 'Chase, BC',
 'White Rock, BC',
 'Maple Ridge, BC',
 'Langley, BC',
 'Salmo, BC',
 'Sun Peaks, BC',
 'Agassiz, BC']

In [9]:
colors = ["royalblue","crimson","lightseagreen","orange","lightgrey"]
scale = 50
limits = [(0, 1), (1, 4), (4, 10), (10, 18), (18, 35)]

fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    df_sub = latlng_df[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(
        lon = df_sub['lng'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = dict(
            # sizeref = 2 * max(latlng_df['count_log']) / (max(latlng_df['count_log']))**3,
            #sizemin = 5,
            size = np.log(df_sub['count']) ** 3.3,
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        hoverinfo='text',
    ))
    

fig.update_layout(
        #title_text = '2014 US city populations<br>(Click legend to toggle traces)', title_x = 0.5,
        showlegend = False,
        geo = dict(
            resolution = 50,
            showcountries=True,
            # visible=False,
            # lataxis = dict(range=[47,53]),
            # lonaxis = dict(range=[-125,-115]),
            landcolor = 'rgb(217, 217, 217)',
            # subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(150, 217, 217)",
            # countrywidth = 0.5,
            # subunitwidth = 0.5
        )
    )

fig.update_geos(
    fitbounds="locations",
    showocean=True, oceancolor="LightBlue",
    showlakes=True, lakecolor="Blue",
    showrivers=True, rivercolor="Blue",
    showcoastlines=True, coastlinecolor="RebeccaPurple",
    showland=True, landcolor="LightGreen",)

fig.show()

In [10]:
colors = ["royalblue","crimson","lightseagreen","orange","lightgrey"]
limits = [(0, 1), (1, 4), (4, 10), (10, 18), (18, 35)]
token = "pk.eyJ1IjoiZ2Jyb3VnaHRvbiIsImEiOiJjbGVvc2dnYjIwNWsyNDFwazgyY2N3azZkIn0.9WneKbc8f8xacgUe8wZHWw"

fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    df_sub = latlng_df[lim[0]:lim[1]]
    fig.add_trace(go.Scattermapbox(
        lon = df_sub['lng'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = go.scattermapbox.Marker(
            size = np.log(df_sub['count']) ** 3.5,
            color = colors[i],
            sizemode = 'area'
        ),
        hoverinfo='text',
    ))

fig.update_layout(
    margin={"r": 250, "t": 20, "l": 250, "b": 20},
    showlegend = False,
    hovermode='closest',
    mapbox=dict(
        accesstoken=token,
        bearing=12,
        center=go.layout.mapbox.Center(
            lat=51.2,
            lon=-122.3
        ),
        pitch=0,
        zoom=4.9
    )
)

fig.show()

In [ ]:
m = folium.Map(location=(50.5, -120.12), zoom_start=7)

folium.CircleMarker(
    location=latlng_dict['Vancouver, BC'],
    radius=60,
    popup="Laurelhurst Park",
    color="#3186cc",
    fill=True,
    fill_color="#3186cc",
).add_to(m)
folium.CircleMarker(
    location=latlng_dict['Nelson, BC'],
    radius=35,
    popup="Laurelhurst Park",
    color="#3186cc",
    fill=True,
    fill_color="#3186cc",
).add_to(m)
folium.CircleMarker(
    location=latlng_dict['New Westminster, BC'],
    radius=20,
    popup="New Westminster, BC",
    color="#3186cc",
    fill=True,
    fill_color="#3186cc",
).add_to(m)
folium.CircleMarker(
    location=latlng_dict['Trail, BC'],
    radius=10,
    popup="Trail, BC",
    color="#3186cc",
    fill=True,
    fill_color="#3186cc",
).add_to(m)
folium.CircleMarker(
    location=latlng_dict['Williams Lake, BC'],
    radius=5,
    popup="Williams Lake, BC",
    color="#3186cc",
    fill=True,
    fill_color="#3186cc",
).add_to(m)
m

In [ ]:
m = folium.Map(location=(50.5, -120.12), zoom_start=7)
folium.GeoJson(os.path.join(CFG.DATA_PATH, 'shapefiles', 'population_centers', 'pop_centers_new.geojson')).add_to(m)
m